## feature_3によってamountの絶対値あたりの価値は違うという仮説で、特徴をリフトにする

In [3]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
import glob
from sklearn.decomposition import PCA
from itertools import combinations

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Load

In [ ]:
#  Data Load
base = utils.read_df_pkl('../input/base_term*0*')
win_path = f'../features/4_winner/*.gz'
win_path_list = glob.glob(win_path)
tmp_path_list = glob.glob('../features/5_tmp/*.gz')
# tmp_path_list = []
win_path_list += tmp_path_list

path_list = win_path_list

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
feature_list = utils.parallel_load_data(path_list=path_list)
df = pd.concat(feature_list, axis=1)

In [7]:
feim = pd.read_csv('../valid/0205_223_valid_lgb_lr0.01_219feats_10seed_68leaves_iter1237_OUT29.8735_CV3.6239445655263887_LB.csv')
top10 = feim['feature'].values[:10]
top10

array(['556_lif_hist_regist_term_lift_206_pst_auth1_cat1_month_diff_mean@',
       '556_lif_hist_regist_term_lift_206_pst_auth1_cat1_monthly_avg_month_diff_mean@',
       '203_pst_new_lag2_diff_date_from_new_max_month_min@',
       '152_ah1__auth1_hist_month_nunique@', '200_ker_hist_duration_min@',
       '203_pst_auth0_yyyymmdd_nunique@',
       '556_lif_hist_regist_term_lift_203_pst_ratio_new_auth1_purchase_amount_sum@',
       '206_pst_auth1_cat1_monthly_avg_month_diff_mean@',
       '556_lif_hist_regist_term_lift_203_pst_auth0_month_diff_mean@',
       '206_pst_auth1_cat1_lag05_diff_date_from_hist_min_month_min@'],
      dtype=object)

In [20]:
#========================================================================
# Regular Featureに対するPCA 
prefix = '222'

pca = PCA(n_components=0.81,whiten=True)

combi_list = combinations(top10, 2)

pca_list = []
for combi in combi_list:
    feature = pca.fit_transform(df[list(combi)].fillna(-1))
    if feature.ravel().shape[0]>500000:
        feat_1 = feature[:, 0]
        feat_2 = feature[:, 1]
        utils.to_pkl_gzip(obj=feat_1, path=f'../features/1_first_valid/{prefix}_PCA_D1_{str(combi)}')
        utils.to_pkl_gzip(obj=feat_2, path=f'../features/1_first_valid/{prefix}_PCA_D2_{str(combi)}')
    else:
        feature = feature.ravel().astype('float32')
        utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_PCA_{str(combi)}')
#========================================================================

In [23]:
#========================================================================
# Regular Featureからは独立していそうなFeatureに対するPCA.
# もしくは、強力だがぶつかってしまうfeatureについて軸を変えてみる
prefix = '223'

pca = PCA(n_components=0.81,whiten=True)

path_list = glob.glob('../features/2_second_valid/558*.gz')
feature_list = utils.parallel_load_data(path_list=path_list)
df = pd.concat(feature_list, axis=1)
combi_list = combinations(df.columns, 2)


for combi in combi_list:
    col0 = combi[0]
    col1 = combi[1]
    feat0 = df[col0]
    feat1 = df[col1]
    if col0.count('date') or col0.count('month_diff'):
        if feat0.min()>=0:
            feat0.fillna(-1, inplace=True)
        else:
            feat0.fillna(feat0.min()-10, inplace=True)
    else:
        feat0.fillna(-1, inplace=True)
        
    if col1.count('date') or col1.count('month_diff'):
        if feat1.min()>=0:
            feat1.fillna(-1, inplace=True)
        else:
            feat1.fillna(feat1.min()-10, inplace=True)
    else:
        feat1.fillna(-1, inplace=True)
        
pca_list = []
impute_list = []
for combi in combi_list:
    try:
        feature = pca.fit_transform(df[list(combi)])
    except ValueError:
        impute_list += list(combi)
        continue
    if feature.ravel().shape[0]>500000:
        feat_1 = feature[:, 0]
        feat_2 = feature[:, 1]
        utils.to_pkl_gzip(obj=feat_1, path=f'../features/1_first_valid/{prefix}_PCA_D1_{str(combi)}')
        utils.to_pkl_gzip(obj=feat_2, path=f'../features/1_first_valid/{prefix}_PCA_D2_{str(combi)}')
    else:
        feature = feature.ravel().astype('float32')
        utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_PCA_{str(combi)}')
#========================================================================

In [14]:
path_list = glob.glob('../features/5_tmp/558*.gz')

In [24]:
for path in impute_list:
    print(path)
#     if utils.read_pkl_gzip(path).shape[0]>330000:
#         print(path)